## 5.4.2 - Groupby Join

In [43]:
import pyspark
from pyspark.sql import SparkSession

In [44]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [45]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [46]:
df_green.createOrReplaceTempView('green')

In [47]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [48]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-22 19:00:00|  49|            163.25|             8|
|2020-01-23 14:00:00| 260|            135.01|             9|
|2020-01-05 20:00:00| 216|            249.89|             7|
|2020-01-16 14:00:00|  82| 309.4800000000002|            30|
|2020-01-17 09:00:00| 116|333.40000000000003|            20|
|2020-01-05 15:00:00|  74|1208.1699999999987|            87|
|2020-01-16 23:00:00|  35|             64.75|             2|
|2020-01-08 21:00:00|  65| 447.3200000000001|            32|
|2020-01-12 05:00:00|  80|            178.57|            11|
|2020-01-22 10:00:00|  74|1390.3599999999988|           101|
|2020-01-04 09:00:00| 260|38.150000000000006|             4|
|2020-01-22 15:00:00|  25| 365.2900000000001|            22|
|2020-01-09 12:00:00| 225|            380.23|            19|
|2020-01-12 15:00:00|  7

In [49]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

24/02/26 21:54:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


24/02/26 21:54:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [50]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [51]:
df_yellow.createOrReplaceTempView('yellow')

In [52]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

In [53]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [54]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

24/02/26 21:55:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


## 5.4.3 - Join in Spark

In [60]:
# Separating files name of amount & number records 
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [63]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')
df_join

DataFrame[hour: timestamp, zone: bigint, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [65]:
# Create total report for both green & yellow
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

24/02/26 22:30:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [64]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  50|              null|                null|             18.36|                    2|
|2020-01-01 00:00:00|  79|              null|                null|             17.76|                    1|
|2020-01-01 00:00:00|  87|              null|                null|             23.16|                    1|
|2020-01-01 00:00:00|  90|              null|                null|             14.12|                    1|
|2020-01-01 00:00:00| 107|              null|                null|              14.3|                    1|
|2020-01-01 00:00:00| 138|              null|                null|            125.94|                    4|
|2020-01-01 00:00:00| 186|  